In [ ]:
import os
import numpy as np
import cv2
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model
import math
import time
from collections import defaultdict

# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load YOLOv7-pose model
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model'].float().eval().to(device)
if torch.cuda.is_available():
    model = model.half()

# Dataset paths (sesuaikan dengan struktur folder Anda)
BASE_PATH = r"C:\Users\LENOVO\Documents\A Skripsi\datasets\FallDataset\Datasets sudah Grouping\A Gelap"
PATHS = {
    'train': {'video': os.path.join(BASE_PATH, 'video', 'train'), 
              'label': os.path.join(BASE_PATH, 'labels', 'train')},
    'valid': {'video': os.path.join(BASE_PATH, 'video', 'valid'),
              'label': os.path.join(BASE_PATH, 'labels', 'valid')},
    'test': {'video': os.path.join(BASE_PATH, 'video', 'test'),
             'label': os.path.join(BASE_PATH, 'labels', 'test')}
}

# ================== Enhanced Fall Detection ==================
def detect_fall(keypoints, threshold=0.3):
    # Keypoint indices
    NOSE = 0
    L_SHOULDER, R_SHOULDER = 5, 6
    L_HIP, R_HIP = 11, 12
    L_ANKLE, R_ANKLE = 15, 16
    
    try:
        # Extract keypoints with lower confidence threshold
        kp = {
            'nose': keypoints[NOSE*3:(NOSE+1)*3],
            'l_shoulder': keypoints[L_SHOULDER*3:(L_SHOULDER+1)*3],
            'r_shoulder': keypoints[R_SHOULDER*3:(R_SHOULDER+1)*3],
            'l_hip': keypoints[L_HIP*3:(L_HIP+1)*3],
            'r_hip': keypoints[R_HIP*3:(R_HIP+1)*3],
            'l_ankle': keypoints[L_ANKLE*3:(L_ANKLE+1)*3],
            'r_ankle': keypoints[R_ANKLE*3:(R_ANKLE+1)*3]
        }
        
        # Check if keypoints are detected
        for k, v in kp.items():
            if v[2] < threshold:
                return False, "low_confidence", ["Keypoint low confidence"]
        
        # Calculate body parameters
        shoulder_y = (kp['l_shoulder'][1] + kp['r_shoulder'][1]) / 2
        feet_y = (kp['l_ankle'][1] + kp['r_ankle'][1]) / 2
        torso_length = np.sqrt((kp['l_shoulder'][0]-kp['l_hip'][0])**2 + 
                         (kp['l_shoulder'][1]-kp['l_hip'][1])**2)
        
        # Enhanced conditions for dark environments
        conditions = []
        
        # 1. Shoulder position relative to feet (adjusted for dark)
        if shoulder_y > feet_y - 0.5 * torso_length:  # More tolerant threshold
            conditions.append("shoulder_near_feet")
        
        # 2. Body orientation
        body_width = abs(kp['l_shoulder'][0] - kp['r_shoulder'][0])
        body_height = abs(kp['nose'][1] - feet_y)
        if body_width / (body_height + 1e-5) > 0.8:  # Reduced threshold
            conditions.append("horizontal_pose")
        
        # 3. Motion detection (fall speed)
        global prev_shoulder_y, prev_frame_time
        if prev_shoulder_y is not None and prev_frame_time is not None:
            time_elapsed = time.time() - prev_frame_time
            if time_elapsed > 0:
                speed = (shoulder_y - prev_shoulder_y) / time_elapsed
                if speed > 0.6:  # More sensitive to fast movements
                    conditions.append(f"high_speed_{speed:.1f}px")
        
        # Update tracking variables
        prev_shoulder_y = shoulder_y
        prev_frame_time = time.time()
        
        # Decision making (at least 2 conditions)
        if len(conditions) >= 2:
            return True, "fallen", conditions
        return False, "normal", conditions
    
    except Exception as e:
        return False, "error", [f"Error: {str(e)}"]

if __name__ == "__main__":
    # Initialize metrics for each phase
    metrics = {
        'train': FallDetectionMetrics(),
        'valid': FallDetectionMetrics(),
        'test': FallDetectionMetrics()
    }
    
    # Process all phases
    for phase in ['train', 'valid', 'test']:
        print(f"\n{'='*40}")
        print(f"Processing {phase} set (Video: {len(os.listdir(PATHS[phase]['video']))} videos)")
        
        # Load annotations
        annotations = load_annotations(PATHS[phase]['label'])
        
        # Process videos
        for video_file in os.listdir(PATHS[phase]['video']):
            if video_file.endswith(".avi"):
                video_path = os.path.join(PATHS[phase]['video'], video_file)
                process_video(video_path, annotations, metrics[phase])
        
        # Save results
        metrics[phase].save_results(f"results_{phase}")
        print(f"{phase} metrics saved to results_{phase}/")
    
    # Print final summary
    print("\n=== FINAL RESULTS ===")
    for phase in ['train', 'valid', 'test']:
        res = metrics[phase].calculate_metrics()
        print(f"\n{phase.upper():<6} Precision: {res['precision']:.3f} | Recall: {res['recall']:.3f} | F1: {res['f1_score']:.3f}")